#### Importação dos dados

In [75]:
import pandas as pd

In [76]:
pokemon = pd.read_csv('./data/pokemon.csv')
types = pd.read_csv('./data/types.csv')[['identifier', 'color']]

#### Tratamento de nulos

In [77]:
pokemon['type2'].fillna(pokemon['type1'], inplace=True)

#### Mescla de tabelas

In [78]:
merged_df = pd.merge(pokemon, types, left_on='type1', right_on='identifier', how='left')
merged_df.rename(columns={'color': 'type1_color'}, inplace=True)

merged_df = pd.merge(merged_df, types, left_on='type2', right_on='identifier', how='left')
merged_df.rename(columns={'color': 'type2_color'}, inplace=True)

# Drop dos identificadores
merged_df = merged_df.drop(['identifier_x', 'identifier_y'], axis=1)

pokemon = merged_df

#### Tranformação dos hex codes em cores rgb

In [79]:
# Função que transforma HEX em RGB
def hex_to_rgb(hex_code):
    hex_code = hex_code.lstrip('#')
    r = int(hex_code[0:2], 16)
    g = int(hex_code[2:4], 16)
    b = int(hex_code[4:6], 16)
    return r, g, b

In [80]:
# Criação de novas colunas para as cores RGB
pokemon['type1_color_rgb'] = pokemon['type1_color'].apply(hex_to_rgb)
pokemon['type2_color_rgb'] = pokemon['type2_color'].apply(hex_to_rgb)

In [81]:
# Criação de novas colunas(Red, Green, Blue) para cada cor
pokemon['type1_color_red'] = pokemon['type1_color_rgb'].apply(lambda x: x[0])
pokemon['type1_color_green'] = pokemon['type1_color_rgb'].apply(lambda x: x[1])
pokemon['type1_color_blue'] = pokemon['type1_color_rgb'].apply(lambda x: x[2])

pokemon['type2_color_red'] = pokemon['type2_color_rgb'].apply(lambda x: x[0])
pokemon['type2_color_green'] = pokemon['type2_color_rgb'].apply(lambda x: x[1])
pokemon['type2_color_blue'] = pokemon['type2_color_rgb'].apply(lambda x: x[2])

In [82]:
# Drop das colunas antigas de cores
pokemon = pokemon.drop(['type1_color','type2_color','type1_color_rgb','type2_color_rgb'], axis=1)

#### Tratamento dos tipos de pokémon

In [83]:
# Criando união dos tipos únicos
unique_types = set(pokemon['type1']).union(set(pokemon['type2']))

In [84]:
# One-hot encoding da coluna de tipos
for type_col in unique_types:
    pokemon[type_col] = (pokemon['type1'] == type_col) | (pokemon['type2'] == type_col)

for type_col in unique_types:
    pokemon[type_col] = pokemon[type_col].astype(int)

In [85]:
# Drop das antigas colunas de tipos 1 e 2
pokemon.drop(['type1', 'type2'], axis=1, inplace=True)